In [ ]:
!pip install scikit-learn torch pandas matplotlib numpy tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd, numpy as np, glob, torch, torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
import math, os

drive_path = '/content/drive/MyDrive/dataverse_files2/*.txt'
save_path = '/content/drive/MyDrive/telecom.csv'


In [ ]:
import pandas as pd
import re

def parse_txt_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # skip empty or metadata lines
            if not line or line.startswith('#'):
                continue

            # split by any whitespace or semicolon
            values = re.split(r'\s+|;', line.strip())

            # only keep rows that have all expected columns (usually 8)
            if len(values) >= 8:
                data.append(values[:8])

    df = pd.DataFrame(
        data,
        columns=[
            'cell_id', 'date', 'time_interval',
            'sms_in', 'sms_out',
            'call_in', 'call_out', 'internet'
        ]
    )
    return df

# Example usage
file_path = '/content/drive/MyDrive/dataverse_files2/sms-call-internet-mi-2013-11-01.txt'
df = parse_txt_file(file_path)
print(df.head())


  cell_id           date time_interval               sms_in  \
0       1  1383260400000            39  0.14186425470242922   
1       1  1383261000000            39  0.27845207746066025   
2       1  1383261600000            39   0.3306414276169333   
3       1  1383262200000            39   0.6814340796890551   
4       1  1383262800000            39  0.24337810266887647   

               sms_out              call_in              call_out  \
0   0.1567870050390246  0.16093793691701822  0.052274848528573205   
1  0.11992572014174135   0.1887771729145041   0.13363747203983203   
2   0.1709520296851148  0.13417624316013174   0.05460092975437236   
3  0.22081529861558874  0.02730046487718618   0.05343788914147278   
4   0.1928905642458027  0.05343788914147278   0.08073835401865896   

             internet  
0  11.028366381681026  
1  11.100963451409388  
2  10.892770602791096  
3   8.622424590989748  
4   8.009927462445756  


In [ ]:
import pandas as pd
import re
import glob
from tqdm import tqdm

# Path to your folder (adjust to your exact Drive path)
path = '/content/drive/MyDrive/dataverse_files2/*.txt'
files = sorted(glob.glob(path))[:20]   # take only the first 20 files

def parse_txt_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            values = re.split(r'\s+|;', line)
            if len(values) >= 8:
                data.append(values[:8])
    df = pd.DataFrame(data, columns=[
        'cell_id', 'date', 'time_interval',
        'sms_in', 'sms_out', 'call_in', 'call_out', 'internet'
    ])
    df['source_file'] = file_path.split('/')[-1]
    return df

dfs = []
for f in tqdm(files):
    try:
        df = parse_txt_file(f)
        if not df.empty:
            dfs.append(df)
    except Exception as e:
        print(f"Error in {f}: {e}")

# Combine all 20 into one DataFrame
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print("Combined shape:", combined_df.shape)
    display(combined_df.head())
else:
    print("No dataframes loaded successfully.")


100%|██████████| 20/20 [09:11<00:00, 27.58s/it]


Combined shape: (21508258, 9)


,cell_id,date,time_interval,sms_in,sms_out,call_in,call_out,internet,source_file
0,1,1383260400000,39,0.14186425470242922,0.1567870050390246,0.16093793691701822,0.052274848528573205,11.028366381681026,sms-call-internet-mi-2013-11-01.txt
1,1,1383261000000,39,0.27845207746066025,0.11992572014174135,0.1887771729145041,0.13363747203983203,11.100963451409388,sms-call-internet-mi-2013-11-01.txt
2,1,1383261600000,39,0.3306414276169333,0.1709520296851148,0.13417624316013174,0.05460092975437236,10.892770602791096,sms-call-internet-mi-2013-11-01.txt
3,1,1383262200000,39,0.6814340796890551,0.22081529861558874,0.02730046487718618,0.05343788914147278,8.622424590989748,sms-call-internet-mi-2013-11-01.txt
4,1,1383262800000,39,0.24337810266887647,0.1928905642458027,0.05343788914147278,0.08073835401865896,8.009927462445756,sms-call-internet-mi-2013-11-01.txt


In [ ]:
combined_df.to_csv('/content/drive/MyDrive/telecom.csv', index=False)


In [ ]:
import os
os.path.exists('/content/drive/MyDrive/telecom.csv')


True